In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/libraries/voxynth')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg

experiment:
  torch_compile: '?' 
  torch_mixed_precision: '?' 

data:
  iters_per_epoch: '?' 
  train_splits: cal
  val_splits: val 

dataloader:
  batch_size: '?'
  num_workers: 1
  pin_memory: True 

optim: # Unclear if we should tune this or not.
  _class: torch.optim.Adam
  lr: '?'

train:
  epochs: '?'
  eval_freq: '?'
  base_pretrained_dir: '?'
  base_checkpoint: 'max-val-dice_score'
  base_pt_select_metric: 'val-dice_score'

loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml log_cfg

log:
  checkpoint_freq: 50
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True
    area_estimation_error:
      _fn: ese.losses.area_estimation_error
      from_logits: True
      square_diff: False

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: 
          - dice_score 
          - area_estimation_error
        phase: val

<IPython.core.display.Javascript object>

In [5]:
%%yaml calibrator_defaults_cfg 


# For calibration methods.
Finetune:
  _class: null
  
TS:
  _class: ese.models.calibrators.Temperature_Scaling

ImageBasedTS:
  _class: ese.models.calibrators.ImageBasedTS
  img_channels: 1
  num_classes: 1

LTS:
  _class: ese.models.calibrators.LocalTS
  img_channels: 1
  num_classes: 1
  dims: '?'
  filters:
    - 8 
    - 8 
    - 8 

3D_LTS:
  _class: ese.models.calibrators.LocalTS
  img_channels: 1
  num_classes: 1
  dims: 3
  filters: [8, 8, 8, 8, 8, 8, 8]

# For calibration methods.
PopcornScaling:
  _class: ese.models.calibrators.Popcorn_Scaling
  ksize: 5 
  init_mode: random 

<IPython.core.display.Javascript object>

In [6]:
%%yaml aug_cfg

augmentations:
    use_mask: False 
    added_noise_probability: 0.75
    added_noise_max_sigma: 0.05
    gamma_scaling_probability: 0.5
    gamma_scaling_max: 0.8
    normalize_min_quantile: 0.98
    normalize_max_scale: 1.02
    bias_field_probability: 0.5

<IPython.core.display.Javascript object>

In [7]:
%%yaml experiment_cfg 

name: "ISLES_3D_TS_wAug"

experiment:
  torch_compile: True 
  torch_mixed_precision: False

train: 
    base_pretrained_dir: "/storage/vbutoi/scratch/ESE/training/09_15_24_ISLES_3D_Dice_wAUG"

data:
    _class: ese.datasets.ISLES

model:
    # class_name: 3D_LTS
    class_name: TS

dataloader:
  batch_size: 4 
  num_workers: 2


loss_func:
    _class: ese.losses.AreaEstimationError

<IPython.core.display.Javascript object>

In [8]:
from ese.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

# Get the configs for the different runs.

# base_cfg = Config(default_cfg).update([log_cfg, callbacks_cfg])
base_cfg = Config(default_cfg).update([log_cfg, callbacks_cfg, aug_cfg])

updated_base_cfg, cal_cfgs = get_ese_calibration_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg,
    calibration_model_cfgs=calibrator_defaults_cfg
)

In [9]:
len(cal_cfgs)

3

## Running Jobs

In [10]:
from ese.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [11]:
# ####### Run individual jobs
# run_ese_exp(
#     config=cal_cfgs[0], 
#     experiment_class=PostHocExperiment,
#     run_name='debug',
#     show_examples=True,
#     track_wandb=False,
#     gpu='2',
# )

In [12]:
#### Run Batch Jobs
submit_ese_exps(
    group="calibration",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=cal_cfgs,
    experiment_class=PostHocExperiment,
    track_wandb=True,
    available_gpus=['2', '3', '4']
)

Submitted job id: 1268538 on gpu: 2.
Submitted job id: 1269281 on gpu: 3.
Submitted job id: 1269799 on gpu: 4.
